In [87]:
import numpy as np
import qml
import unittest
import itertools as it

In [79]:
class Networker():
    def __init__(self, filename):
        self._c = qml.Compound(filename)
        self._atoms = np.where(self._c.nuclear_charges == 6)[0]
    
    def get_similarity(self, nuclear_charges):
        """ Returns i, j, distance."""
        charges = self._c.nuclear_charges.copy().astype(np.float)
        charges[self._atoms] = nuclear_charges
        atoms = np.where(self._c.nuclear_charges == 6)[0]
        a = qml.representations.generate_coulomb_matrix(charges, self._c.coordinates, size=self._c.natoms, sorting='unsorted')
        s = np.zeros((self._c.natoms, self._c.natoms))
        s[np.tril_indices(self._c.natoms)] = a
        d = np.diag(s)
        s += s.T
        s[np.diag_indices(self._c.natoms)] = d
        sorted_elements = [np.sort(_) for _ in s[atoms]]
        ret = []
        for i in range(len(atoms)):
            for j in range(i+1, len(atoms)):
                dist = np.linalg.norm(sorted_elements[i] - sorted_elements[j])
                ret.append([atoms[i], atoms[j], dist])
        return ret, atoms
    
    def identify_equivalent(self, nuclear_charges):
        """ Returns groups of equivalent sites. """
        similarities, relevant = self.get_similarity(nuclear_charges)
        groups = []
        placed = []
        for i, j, dist in similarities:
            if dist > 0.1:
                continue
            for gidx, group in enumerate(groups):
                if i in group:
                    if j not in group:
                        groups[gidx].append(j)
                        placed.append(j)
                    break
                if j in group:
                    if i not in group:
                        groups[gidx].append(i)
                        placed.append(i)
                    break
            else:
                groups.append([i,j])
                placed += [i, j]
        for isolated in set(relevant) - set(placed):
            groups.append([isolated])
        return groups

In [85]:
## Tests
class TestNetworker(unittest.TestCase):
    def _run_test(self, zs, expected):
        n = Networker("../../test/benzene.xyz")
        actual = n.identify_equivalent(zs)
        self.assertEqual(actual, expected)
        
    def test_all_identical(self):
        self._run_test(np.array([6,6,6,6,6,6]), [[0, 1, 2, 3, 4, 5]])
        
    def test_one_different(self):
        self._run_test(np.array([6,6,6,6,6,5]), [[0, 4], [1, 3], [2], [5]])
        
    def test_fractional(self):
        self._run_test(np.array([6,6,6,6,6,5.5]),[[0, 4], [1, 3], [2], [5]])
        
    def test_fractional_groups(self):
        self._run_test(np.array([6,6,6,6,5.9,5.9]),[[0, 3], [1, 2], [4, 5]])
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_all_identical (__main__.TestNetworker) ... ok
test_fractional (__main__.TestNetworker) ... ok
test_fractional_groups (__main__.TestNetworker) ... ok
test_one_different (__main__.TestNetworker) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.197s

OK


In [118]:
def targets_from_reference(reference):
    pass
def check_common_ground(target, opposite, reference, common_ground):
    deltaZ = opposite - target
    
    # matching deltaZ
    values, counts = np.unique(deltaZ, return_counts=True)
    counts = dict(zip(values, counts))
    
    for value in values:
        if value == 0:
            continue
        if -value not in counts:
            return False
        if counts[-value] != counts[value]:
            return False
    
    # ignore id operation
    if max(np.abs(deltaZ)) == 0:
        return False
    
    # all changing atoms need to be in the same group
    for value in values:
        if value <= 0:
            continue
            
        changed = np.where(np.abs(deltaZ))[0]
        for group in common_ground:
            if len(set(changed) - set(group)) == 0:
                break
        else:
            return False
        
    print (deltaZ, common_ground, )
    return True
    
def references_from_target(networker, target):
    target = np.array(target)
    
    # get all possible opposite targets
    candidates = set([_ for _ in it.permutations(target)])
    candidates = [np.array(_) for _ in candidates]
    
    found = []
    for opposite in candidates:
        reference = (opposite + target) / 2
        common_ground = networker.identify_equivalent(reference)
        if check_common_ground(target, opposite, reference, common_ground):
            found.append((reference, opposite))
    return found

In [119]:
n = Networker("../../test/benzene.xyz")
#n.identify_equivalent(np.array([5,7,6,6,6,6]))
references_from_target(n, [5,7,6,6,6,6])

[ 0 -1  0  0  0  1] [[1, 5], [2, 4], [0], [3]]
[ 1  0 -1  0  0  0] [[0, 2], [3, 5], [1], [4]]
[ 2 -2  0  0  0  0] [[0, 1, 2, 3, 4, 5]]


[(array([5. , 6.5, 6. , 6. , 6. , 6.5]), array([5, 6, 6, 6, 6, 7])),
 (array([5.5, 7. , 5.5, 6. , 6. , 6. ]), array([6, 7, 5, 6, 6, 6])),
 (array([6., 6., 6., 6., 6., 6.]), array([7, 5, 6, 6, 6, 6]))]

In [92]:
# BN CCCC
np.array(set([_ for _ in it.permutations(np.array([1,2,1.1]))]))

array({(1.0, 2.0, 1.1), (1.1, 2.0, 1.0), (2.0, 1.0, 1.1), (2.0, 1.1, 1.0), (1.1, 1.0, 2.0), (1.0, 1.1, 2.0)},
      dtype=object)

In [113]:
set((1,2, 4)) - set((1,2,3))

{4}